In [4]:
import os.path  # For checking the existence of files
import re  # For regular expression operations
import pandas as pd  # For handling CSV operations
from google.auth.transport.requests import Request  # For refreshing credentials
from google.oauth2.credentials import Credentials  # For handling OAuth2 credentials
from google_auth_oauthlib.flow import InstalledAppFlow  # For handling the OAuth2 flow
from googleapiclient.discovery import build  # For building the Gmail API service

# Define the scope for accessing Gmail. This scope allows read-only access to Gmail.
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']

def authenticate_gmail():
    """
    Authenticates the user with Gmail API and returns the credentials.
    """
    creds = None  # Initialize credentials to None
    # Load credentials from token.json if it exists
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    # If there are no valid credentials, prompt the user to log in
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())  # Refresh the credentials if they are expired
        else:
            # Start the OAuth2 flow to get new credentials
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json',
                SCOPES,
                redirect_uri='https://49f4-184-145-46-31.ngrok-free.app/oauth2callback'  # Update with ngrok URL
            )
            creds = flow.run_local_server(port=8080)  # Run the local server for user authentication
        # Save the credentials for future use
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    return creds  # Return the authenticated credentials

def search_emails(service, query):
    """
    Searches emails in the Gmail account based on the query and extracts email addresses from the 'To' field.
    """
    results = service.users().messages().list(userId='me', q=query).execute()  # Search for emails matching the query
    messages = results.get('messages', [])  # Get the list of messages

    email_addresses = set()  # Initialize a set to store unique email addresses

    if not messages:  # If no messages are found, print a message
        print('No messages found.')
    else:
        for msg in messages:  # Iterate through the found messages
            msg_id = msg['id']  # Get the message ID
            message = service.users().messages().get(userId='me', id=msg_id).execute()  # Get the full message
            payload = message['payload']  # Get the payload of the message
            headers = payload.get('headers', [])  # Get the headers of the message
            for header in headers:  # Iterate through the headers
                if header['name'] == 'To':  # If the header is 'To', process the email addresses
                    emails = header['value']  # Get the value of the 'To' header
                    email_list = re.findall(r'\b[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,}\b', emails, re.I)  # Find all email addresses using regex
                    email_addresses.update(email_list)  # Add the found email addresses to the set
    
    return email_addresses  # Return the set of email addresses

def save_to_csv(email_addresses, filename='email_addresses.csv'):
    """
    Saves the set of email addresses to a CSV file.
    """
    df = pd.DataFrame(list(email_addresses), columns=['Email Address'])  # Create a DataFrame from the email addresses
    df.to_csv(filename, index=False)  # Save the DataFrame to a CSV file
    print(f"Email addresses saved to {filename}")  # Print a message indicating the file has been saved

def main():
    """
    Main function to authenticate and search for emails containing a specific word, then save the results to a CSV file.
    """
    creds = authenticate_gmail()  # Authenticate the user and get the credentials
    service = build('gmail', 'v1', credentials=creds)  # Build the Gmail API service

    # Modify 'specific_word' to the word you want to search for in the subject field of the sent emails.
    specific_word = 'server'  # The word to search for in the email subjects
    query = f'subject:{specific_word} in:sent'  # The query to search for emails with the word in the subject and in the sent folder

    email_addresses = search_emails(service, query)  # Search for the emails and get the email addresses
    
    print("Email Addresses found:")  # Print the found email addresses
    for email in email_addresses:
        print(email)
    
    # Save the email addresses to a CSV file
    save_to_csv(email_addresses)  # Save the email addresses to a CSV file

if __name__ == '__main__':
    main()  # Execute the main function if the script is run directly


Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=887556623145-5gnij0j9suo9558jcdt6do15ac0sao8b.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.readonly&state=zbMyp1VFRkkAJcAaqsiyndyWZFdvuh&access_type=offline
Email Addresses found:
info@mercimonami.ca
piebar@fabrestaurants.ca
cm@chinchin.com
info@miracletoronto.com
goodman@fabrestaurants.ca
info@overpressureclub.com
hr@oliverbonacini.com
feedback@mrgreek.com
customerservice@kernelspopcorn.com
wanglangtoronto@gmail.com
catering@milagrorestaurant.com
echokaraoke.onbloor@gmail.com
info@alicetoronto.com
gigionbloor@gmail.com
info@thehandlebar.ca
info@sneaky-dees.com
themorningafter88@gmail.com
Kristen@paradisegrapevine.com
info@alobaryorkville.com
industry.snack.bar@gmail.com
hello@cockadoodledoo.ca
contact@snakesandlattes.com
info@misterctoronto.com
info@jzspizza.com
254bsmt@gmail.com
ci

In [14]:
import os
import base64
import pandas as pd
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build

# Define the scope for accessing and sending Gmail
SCOPES = ['https://www.googleapis.com/auth/gmail.send']

def authenticate_gmail():
    """
    Authenticates the user with Gmail API and returns the credentials.
    """
    creds = None  # Initialize credentials to None
    # Load credentials from token.json if it exists
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    # If there are no valid credentials, prompt the user to log in
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())  # Refresh the credentials if they are expired
        else:
            # Start the OAuth2 flow to get new credentials
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES, redirect_uri='https://49f4-184-145-46-31.ngrok-free.app/oauth2callback')
            creds = flow.run_local_server(port=8080)  # Run the local server for user authentication
        # Save the credentials for future use
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    return creds  # Return the authenticated credentials

def create_message(sender, to, subject, message_text, is_html=False, attachment_file=None):
    """
    Create an email message with optional HTML body and attachment.
    
    Parameters:
    - sender: Email address of the sender
    - to: Email address of the recipient
    - subject: Subject of the email
    - message_text: Body of the email (plain text or HTML)
    - is_html: Boolean flag to indicate if the message_text is HTML
    - attachment_file: Path to the file to attach (if any)
    
    Returns:
    - dict: The encoded email message ready for sending via Gmail API
    """
    message = MIMEMultipart()
    message['to'] = to
    message['from'] = sender
    message['subject'] = subject

    # Add body text
    if is_html:
        message.attach(MIMEText(message_text, 'html'))
    else:
        message.attach(MIMEText(message_text, 'plain'))

    # Attach file
    if attachment_file:
        part = MIMEBase('application', 'octet-stream')
        with open(attachment_file, 'rb') as file:
            part.set_payload(file.read())
        encoders.encode_base64(part)
        part.add_header(
            'Content-Disposition',
            'attachment; filename="{}"'.format(os.path.basename(attachment_file))
        )
        message.attach(part)

    raw = base64.urlsafe_b64encode(message.as_bytes())
    raw = raw.decode()
    return {'raw': raw}

def send_message(service, user_id, message):
    """
    Send an email message.
    """
    try:
        message = service.users().messages().send(userId=user_id, body=message).execute()
        print(f'Message Id: {message["id"]}')
        return message
    except Exception as error:
        print(f'An error occurred: {error}')
        return None

def main():
    """
    Main function to send personalized emails with attachments.
    """
    creds = authenticate_gmail()
    service = build('gmail', 'v1', credentials=creds)

    html_content = """\
    <html>
    <head>
        <style>
            body { font-family: Arial, sans-serif; line-height: 1.6; }
            h1, h2, h3 { color: #333; }
            p { margin: 0 0 1em; }
            .contact-info { margin-bottom: 2em; }
            .key-skills { margin: 1em 0; }
            .key-skills ul { list-style: none; padding: 0; }
            .key-skills li { margin-bottom: 0.5em; }
        </style>
    </head>
    <body>
        <div class="contact-info">
            <p>Rifatul Islam Majumder<br>
            Toronto, ON M6J 1N8, Canada<br>
            437-566-9332<br>
            rifatulislam210@gmail.com<br>
            July 28, 2024</p>
        </div>

        <p>Dear Hiring Manager,</p>

        <p>I am writing to express my interest in the [Server/Host/Support] position at your Company. With a diverse background in customer service, a strong technical skill set, and a commitment to delivering exceptional service, I am confident in my ability to contribute positively to your team.</p>

        <p>In my role as a Customer Service Representative at Pizza Hut, I developed strong communication and organizational skills. I handled customer inquiries, managed orders, and ensured customer satisfaction. This experience has honed my ability to multitask and work efficiently in a fast-paced environment, making me well-suited for the demands of a [Server/Host/Support] role.</p>

        <p>My academic background in biotechnology from Seneca College and pharmaceuticals from East West University has equipped me with critical thinking and problem-solving abilities. Additionally, my proficiency in various programming languages, including Python and SQL, and my experience with data analysis and web development, allow me to bring a unique, tech-savvy perspective to any support role. I can leverage these skills to streamline processes, enhance efficiency, and ensure the highest level of service.</p>

        <div class="key-skills">
            <h2>Key Skills and Attributes:</h2>
            <ul>
                <li><strong>Customer Service Excellence:</strong> Proven ability to manage customer interactions with a focus on satisfaction and retention.</li>
                <li><strong>Technical Proficiency:</strong> Skilled in data analysis, programming, and using various software tools.</li>
                <li><strong>Organizational Skills:</strong> Strong multitasking abilities, ensuring efficient and effective task management.</li>
                <li><strong>Communication:</strong> Excellent interpersonal and communication skills, essential for both customer-facing and team collaboration roles.</li>
                <li><strong>Problem-Solving:</strong> Adept at identifying issues and implementing solutions to improve operations.</li>
            </ul>
        </div>

        <p>During my tenure at MST Rebar as a Molder & Cutter, I demonstrated my ability to manage precise tasks and maintain high-quality standards. These roles required meticulous attention to detail and the ability to follow strict procedures, which are crucial skills for a [Server/Host/Support] position. Additionally, my First Aid and CPR certification underscores my commitment to maintaining a safe and secure environment for both customers and colleagues.</p>

        <p>I am particularly drawn to your Company because of your reputation for excellent service and your dynamic work environment. I am eager to bring my diverse skill set and dedication to your team, ensuring a seamless and positive experience for your customers.</p>

        <p>Thank you for considering my application. I look forward to the opportunity to discuss how my background, skills, and enthusiasm align with the needs of your Company. Please feel free to contact me at 437-566-9332 or via email at rifatulislam210@gmail.com to schedule an interview.</p>

        <p>Best regards,<br>
        Rifatul Islam Majumder</p>
    </body>
    </html>
    """

    sender = "rifatulislam210@gmail.com"
    subject = "Application for Server/Host/Support Position"
    attachment_file = 'resume_combined.docx'  # Path to your resume file
    #read csv file
    email_data = pd.read_csv('email_addresses.csv')
    for index, row in email_data.iterrows():
        to = row['Email Address']
        message = create_message(sender, to, subject, html_content, is_html=True, attachment_file=attachment_file)
        send_message(service, 'me', message)

    # to = 'rifatulislam212@gmail.com'
    # message = create_message(sender, to, subject, html_content, is_html=True, attachment_file=attachment_file)
    # send_message(service, 'me', message)

if __name__ == '__main__':
    main()


Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=887556623145-5gnij0j9suo9558jcdt6do15ac0sao8b.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.send&state=nrvHq49832k2VIFUTzZYjw1ltUEL9C&access_type=offline
Message Id: 190facade28c2766
Message Id: 190facae03e21cd0
Message Id: 190facae3e0a99bf
Message Id: 190facae528e714b
Message Id: 190facae78b9b5a5
Message Id: 190facae802f7c39
Message Id: 190facaedf473c0a
Message Id: 190facaef3929a70
Message Id: 190facaf117fd5da
Message Id: 190facaf27757229
Message Id: 190facaf4cdb3ceb
Message Id: 190facaf826dc73d
Message Id: 190facafadd7b399
Message Id: 190facaff606449f
Message Id: 190facb01a49ea59
Message Id: 190facb0a93ef32d
Message Id: 190facb100f725bd
Message Id: 190facb14dcceebf
Message Id: 190facb18a04698a
Message Id: 190facb20567e9ee
Message Id: 190facb2ba81565d
Message Id: 190facb3701fbf3f
Mes